In [1]:
#nn模块构建于autograd模块之上，模块化了一些结构
#首先需要继承nn.Moudle,并实现forward

In [2]:
import torch.nn as nn
import torch.nn.functional as Func

class LeNet(nn.Module):
    def __init__(self):
        #nn.module的子类必须在构造函数中执行父类的构造函数
        #下面的等价于nn.Module.__init__(self)
        super(LeNet,self).__init__()
        #卷基层'1'输入图片为单通道图片，‘6’为输出通道数，'5'表示卷积核5*5
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        #全链接层，y=wh+b
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        #卷积->激活->池化 
        x = Func.max_pool2d(Func.relu(self.conv1(x)),(2,2))
        x = Func.max_pool2d(Func.relu(self.conv2(x)),2)
        #把x重新排列，默认-1
        x = x.view(x.size()[0],-1)
        x = Func.relu(self.fc1(x))
        x = Func.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
net = LeNet()
print(net)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
#可通过net.parameters()获得参数
params = list(net.parameters())
#不要直接打印出来，因为会把所有信息都打印出来
print(len(params))
#循环遍历出
for name,param in net.named_parameters():
    print(name,':',param.size())

10
conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


In [5]:
#只有variable才有自动求导功能，Tensor不可以
#所以输入要先变成variable形式
from torch.autograd import Variable
import torch as t
input = Variable(t.randn(1,1,32,32))
out = net(input)
out.size()

net.zero_grad()#clear
out.backward(Variable(t.ones(1,10)))